In [100]:
import json
import pandas as pd

### First prepare the json so that the rating is formatted better

In [101]:
with open('completed_perfumes.json', 'r', encoding='utf-8') as f:
    big_dic = json.load(f)


for idx, list_element in enumerate(big_dic):

    detailed_ratings = big_dic[idx]['detailed_ratings']

    for typ, rating_list in detailed_ratings.items():
        buf_dic = {}
        if rating_list is not None:
            for rating_element in rating_list:
                buf_dic[rating_element['r']] = rating_element['v']
            detailed_ratings[typ] = buf_dic
        else:
            detailed_ratings[typ] = None 


    big_dic[idx]['detailed_ratings'] = detailed_ratings

df = pd.json_normalize(big_dic)

**Take a look at an example element**

In [102]:
df[df['name'] == 'Khamrah']

,name,brand,year,collection,flakon_designer,perfumer,top_notes,middle_notes,base_notes,all_notes,url,ratings.Duft.rating,ratings.Duft.number_of_ratings,ratings.Haltbarkeit.rating,ratings.Haltbarkeit.number_of_ratings,ratings.Sillage.rating,ratings.Sillage.number_of_ratings,ratings.Flakon.rating,ratings.Flakon.number_of_ratings,ratings.Preis-Leistungs-Verhältnis.rating,ratings.Preis-Leistungs-Verhältnis.number_of_ratings,detailed_ratings.scent.0,detailed_ratings.scent.1,detailed_ratings.scent.2,detailed_ratings.scent.3,detailed_ratings.scent.4,detailed_ratings.scent.5,detailed_ratings.scent.6,detailed_ratings.scent.7,detailed_ratings.scent.8,detailed_ratings.scent.9,detailed_ratings.scent.10,detailed_ratings.durability.0,detailed_ratings.durability.1,detailed_ratings.durability.2,detailed_ratings.durability.3,detailed_ratings.durability.4,detailed_ratings.durability.5,detailed_ratings.durability.6,detailed_ratings.durability.7,detailed_ratings.durability.8,detailed_ratings.durability.9,detailed_ratings.durability.10,detailed_ratings.sillage.0,detailed_ratings.sillage.1,detailed_ratings.sillage.2,detailed_ratings.sillage.3,detailed_ratings.sillage.4,detailed_ratings.sillage.5,detailed_ratings.sillage.6,detailed_ratings.sillage.7,detailed_ratings.sillage.8,detailed_ratings.sillage.9,detailed_ratings.sillage.10,detailed_ratings.bottle.0,detailed_ratings.bottle.1,detailed_ratings.bottle.2,detailed_ratings.bottle.3,detailed_ratings.bottle.4,detailed_ratings.bottle.5,detailed_ratings.bottle.6,detailed_ratings.bottle.7,detailed_ratings.bottle.8,detailed_ratings.bottle.9,detailed_ratings.bottle.10,detailed_ratings.pricing.0,detailed_ratings.pricing.1,detailed_ratings.pricing.2,detailed_ratings.pricing.3,detailed_ratings.pricing.4,detailed_ratings.pricing.5,detailed_ratings.pricing.6,detailed_ratings.pricing.7,detailed_ratings.pricing.8,detailed_ratings.pricing.9,detailed_ratings.pricing.10,scent_types.Zitrus,scent_types.Würzig,scent_types.Grün,scent_types.Fruchtig,scent_types.Frisch,scent_types.Blumig,scent_types.Herren,scent_types.Damen,scent_types.Klassisch,scent_types.Modern,scent_types.Winter,scent_types.Herbst,scent_types.Sommer,scent_types.Frühling,scent_types.Täglich,scent_types.Sport,scent_types.Freizeit,scent_types.Ausgehen,scent_types.Arbeit,scent_types.Abend,detailed_ratings.pricing,scent_types.Synthetisch,scent_types.Süß,scent_types.Holzig,scent_types.Aquatisch,scent_types.Gourmand,scent_types.Pudrig,scent_types.Orientalisch,scent_types.Cremig,scent_types.Ledrig,scent_types.Harzig,scent_types.Rauchig,scent_types.Erdig,scent_types.Animalisch,scent_types.Fougère,scent_types.Chypre,detailed_ratings.durability,detailed_ratings.sillage,detailed_ratings.bottle,detailed_ratings.scent
992,Khamrah,Lattafa / لطافة,2022.0,None,None,None,"[Zimt, Muskat, Bergamotte]","[Dattel, Praline, Mahonial®, Tuberose]","[geröstete Tonkabohne, Vanille, Amberholz, Sia...",None,https://www.parfumo.de/Parfums/Lattafa/khamrah,8.1,2818.0,8.5,2681.0,8.2,2666.0,8.2,2628.0,9.3,2587.0,8.0,22.0,23.0,39.0,46.0,78.0,145.0,459.0,907.0,725.0,366.0,2.0,1.0,2.0,6.0,11.0,19.0,63.0,245.0,998.0,854.0,480.0,2.0,2.0,2.0,2.0,15.0,28.0,87.0,382.0,1144.0,655.0,347.0,3.0,7.0,5.0,16.0,27.0,62.0,177.0,350.0,799.0,608.0,574.0,2.0,4.0,2.0,6.0,11.0,23.0,40.0,86.0,300.0,583.0,1530.0,NaN,16.18,NaN,NaN,NaN,NaN,54.96,45.04,40.0,60.0,49.43,41.89,NaN,8.67,9.22,NaN,17.77,32.38,5.23,35.39,NaN,NaN,26.65,5.37,NaN,24.25,NaN,17.51,10.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Remove the arabic letters to avoid formatting issues**

In [104]:
# strip of arabic name to have it cleaner
df['brand'] = df['brand'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()
df[df['name'] == 'Khamrah']

,name,brand,year,collection,flakon_designer,perfumer,top_notes,middle_notes,base_notes,all_notes,url,ratings.Duft.rating,ratings.Duft.number_of_ratings,ratings.Haltbarkeit.rating,ratings.Haltbarkeit.number_of_ratings,ratings.Sillage.rating,ratings.Sillage.number_of_ratings,ratings.Flakon.rating,ratings.Flakon.number_of_ratings,ratings.Preis-Leistungs-Verhältnis.rating,ratings.Preis-Leistungs-Verhältnis.number_of_ratings,detailed_ratings.scent.0,detailed_ratings.scent.1,detailed_ratings.scent.2,detailed_ratings.scent.3,detailed_ratings.scent.4,detailed_ratings.scent.5,detailed_ratings.scent.6,detailed_ratings.scent.7,detailed_ratings.scent.8,detailed_ratings.scent.9,detailed_ratings.scent.10,detailed_ratings.durability.0,detailed_ratings.durability.1,detailed_ratings.durability.2,detailed_ratings.durability.3,detailed_ratings.durability.4,detailed_ratings.durability.5,detailed_ratings.durability.6,detailed_ratings.durability.7,detailed_ratings.durability.8,detailed_ratings.durability.9,detailed_ratings.durability.10,detailed_ratings.sillage.0,detailed_ratings.sillage.1,detailed_ratings.sillage.2,detailed_ratings.sillage.3,detailed_ratings.sillage.4,detailed_ratings.sillage.5,detailed_ratings.sillage.6,detailed_ratings.sillage.7,detailed_ratings.sillage.8,detailed_ratings.sillage.9,detailed_ratings.sillage.10,detailed_ratings.bottle.0,detailed_ratings.bottle.1,detailed_ratings.bottle.2,detailed_ratings.bottle.3,detailed_ratings.bottle.4,detailed_ratings.bottle.5,detailed_ratings.bottle.6,detailed_ratings.bottle.7,detailed_ratings.bottle.8,detailed_ratings.bottle.9,detailed_ratings.bottle.10,detailed_ratings.pricing.0,detailed_ratings.pricing.1,detailed_ratings.pricing.2,detailed_ratings.pricing.3,detailed_ratings.pricing.4,detailed_ratings.pricing.5,detailed_ratings.pricing.6,detailed_ratings.pricing.7,detailed_ratings.pricing.8,detailed_ratings.pricing.9,detailed_ratings.pricing.10,scent_types.Zitrus,scent_types.Würzig,scent_types.Grün,scent_types.Fruchtig,scent_types.Frisch,scent_types.Blumig,scent_types.Herren,scent_types.Damen,scent_types.Klassisch,scent_types.Modern,scent_types.Winter,scent_types.Herbst,scent_types.Sommer,scent_types.Frühling,scent_types.Täglich,scent_types.Sport,scent_types.Freizeit,scent_types.Ausgehen,scent_types.Arbeit,scent_types.Abend,detailed_ratings.pricing,scent_types.Synthetisch,scent_types.Süß,scent_types.Holzig,scent_types.Aquatisch,scent_types.Gourmand,scent_types.Pudrig,scent_types.Orientalisch,scent_types.Cremig,scent_types.Ledrig,scent_types.Harzig,scent_types.Rauchig,scent_types.Erdig,scent_types.Animalisch,scent_types.Fougère,scent_types.Chypre,detailed_ratings.durability,detailed_ratings.sillage,detailed_ratings.bottle,detailed_ratings.scent
992,Khamrah,Lattafa,2022.0,None,None,None,"[Zimt, Muskat, Bergamotte]","[Dattel, Praline, Mahonial®, Tuberose]","[geröstete Tonkabohne, Vanille, Amberholz, Sia...",None,https://www.parfumo.de/Parfums/Lattafa/khamrah,8.1,2818.0,8.5,2681.0,8.2,2666.0,8.2,2628.0,9.3,2587.0,8.0,22.0,23.0,39.0,46.0,78.0,145.0,459.0,907.0,725.0,366.0,2.0,1.0,2.0,6.0,11.0,19.0,63.0,245.0,998.0,854.0,480.0,2.0,2.0,2.0,2.0,15.0,28.0,87.0,382.0,1144.0,655.0,347.0,3.0,7.0,5.0,16.0,27.0,62.0,177.0,350.0,799.0,608.0,574.0,2.0,4.0,2.0,6.0,11.0,23.0,40.0,86.0,300.0,583.0,1530.0,NaN,16.18,NaN,NaN,NaN,NaN,54.96,45.04,40.0,60.0,49.43,41.89,NaN,8.67,9.22,NaN,17.77,32.38,5.23,35.39,NaN,NaN,26.65,5.37,NaN,24.25,NaN,17.51,10.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**remove some unnecessary parts in the column names**

In [105]:
df = df.drop(df.filter(regex='^scent_strength').columns, axis=1)

df.columns = df.columns.str.replace(r'^[^.]+\.', '', regex=True)

print(df.columns)


Index(['name', 'brand', 'year', 'collection', 'flakon_designer', 'perfumer', 'top_notes', 'middle_notes', 'base_notes', 'all_notes',
       ...
       'Harzig', 'Rauchig', 'Erdig', 'Animalisch', 'Fougère', 'Chypre', 'durability', 'sillage', 'bottle', 'scent'],
      dtype='object', length=116)


**Save to excel**

In [99]:
df.to_excel("perfumes.xlsx", sheet_name="Sheet1", index=False)


In [107]:
KRITERIUM = 'Würzig'
ANZ_DUFT_RATINGS = 1000
crit =  KRITERIUM 
duft = 'Duft' + '.rating'
duft_ratings = 'Duft' + '.number_of_ratings'
haltbarkeit = 'Haltbarkeit' + '.rating'
sillage = 'Sillage' + '.rating'

filtered_df = df[
    (df[duft] > 7.5) & 
    (df[haltbarkeit] > 7.5) & 
    (df[sillage] > 7.5) & 
    (df[duft_ratings] > ANZ_DUFT_RATINGS)
    ].sort_values(by=crit, ascending=False)

# Select specific columns and show the first 10 rows
result = filtered_df[['name', 'brand', crit, duft, haltbarkeit, sillage]].head(20)

# Print result
pd.set_option('display.width', 250)  # Set the max width (adjust as needed)
pd.set_option('display.max_columns', None)  # Ensure all columns are displayed
print(result)

                                                   name                brand  Würzig  Duft.rating  Haltbarkeit.rating  Sillage.rating
3383                                  Spicebomb Extreme         Viktor  Rolf   32.36          8.4                 8.4             8.3
3150                                     Sauvage Elixir                 Dior   28.11          8.1                 9.1             8.8
381                                         Intoxicated               Kilian   25.86          8.2                 8.1             7.7
883                                   Casamorati - 1888              XerJoff   25.57          8.2                 8.4             7.9
2972                                            Haltane     Parfums de Marly   24.63          8.3                 8.3             8.0
659       African Leather Memo Paris 2015 Eau de Parfum           Memo Paris   24.14          8.4                 8.4             8.0
2553                                              Rehab       

---

### Now we can think about how to choose the right features and what to analyize

In [37]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

df['base_notes'] = df['base_notes'].apply(lambda x: x if isinstance(x, list) else [])
df['middle_notes'] = df['middle_notes'].apply(lambda x: x if isinstance(x, list) else [])
df['top_notes'] = df['top_notes'].apply(lambda x: x if isinstance(x, list) else [])
df['all_notes'] = df['all_notes'].apply(lambda x: x if isinstance(x, list) else [])

# Apply MultiLabelBinarizer to each column and create new DataFrames
base_encoded = pd.DataFrame(mlb.fit_transform(df['base_notes']), columns=mlb.classes_, index=df.index)
middle_encoded = pd.DataFrame(mlb.fit_transform(df['middle_notes']), columns=mlb.classes_, index=df.index)
top_encoded = pd.DataFrame(mlb.fit_transform(df['top_notes']), columns=mlb.classes_, index=df.index)
all_encoded = pd.DataFrame(mlb.fit_transform(df['all_notes']), columns=mlb.classes_, index=df.index)

# Merge with the original DataFrame
df_encoded = pd.concat([df, base_encoded, middle_encoded, top_encoded, all_encoded], axis=1)

In [45]:
base_encoded = pd.DataFrame(
    mlb.fit_transform(df['base_notes']), 
    columns=[f'base_{col}' for col in mlb.classes_], 
    index=df.index
)

middle_encoded = pd.DataFrame(
    mlb.fit_transform(df['middle_notes']), 
    columns=[f'middle_{col}' for col in mlb.classes_], 
    index=df.index
)

top_encoded = pd.DataFrame(
    mlb.fit_transform(df['top_notes']), 
    columns=[f'top_{col}' for col in mlb.classes_], 
    index=df.index
)

# Merge everything into a single DataFrame
df_encoded = pd.concat([df, base_encoded, middle_encoded, top_encoded], axis=1)

TypeError: 'NoneType' object is not iterable

In [40]:
df_encoded['Bergamotte']==1

,Bergamotte,Bergamotte,Bergamotte,Bergamotte
0,False,False,True,False
1,False,False,True,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
3989,False,False,False,False
3990,False,False,False,False
3991,False,False,False,False
3992,False,False,False,False


In [75]:
df_old['scent_types.Abend'] != df_extension['scent_types.Abend']

ValueError: Can only compare identically-labeled Series objects

In [76]:
print(df_old.index.equals(df_extension.index))  # Should return True

False


In [77]:
df_old = df_old.reset_index(drop=True)
df_extension = df_extension.reset_index(drop=True)

In [78]:
df_old['scent_types.Abend'] != df_extension['scent_types.Abend']

ValueError: Can only compare identically-labeled Series objects

In [79]:
print(df_old.shape, df_extension.shape)  # Should be identical

(4252, 74) (3994, 116)
